# Make Firm List Searchable

In [1]:
import pandas as pd
import pickle

In [2]:
# Open Up List of Fuzzy Matched firms and the orginal databases
OAGlist = pd.read_pickle('OAGlist.pkl')
# Open the orginal databases
CFPB = pd.read_csv('Consumer_Complaints.csv')
CFPB = CFPB[CFPB.State == 'NY']
silvercase = pd.read_csv("OAG Complaints-Online_Final.csv")

/Users/maxfacts/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (5,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Make the firm names lower case
lowfirm = []
lowcusp = []
for firm in OAGlist.firm:
    lowfirm.append(firm.lower())
for firm in OAGlist.cusp_firm_name:
    lowcusp.append(firm.lower())
OAGlist['firm'] = lowfirm
OAGlist['cusp_firm_name'] = lowcusp

In [4]:
#Add a length of each word column to the dataframe
#selected_relevent_tweet = pickle.load(open('selected_relevent_tweet.pkl', 'rb'))
firmlist = OAGlist
length = []
for firm in firmlist.firm:
    temp = len(firm)
    length.append(temp)
#firmlist = firmlist.firm.dropna()
firmlist['length'] = length
print len(firmlist)

18709


In [5]:
#Remove all firm names that are less than 4 charters long.  
#Mannually re-add firms that were arbitrarly selected to remain that were
#less than 4 chareters long
firmlist = firmlist[firmlist.length > 4]
manualadd = ['irs', 'adt', 'cus', 'ups', 'qvc', 'aol', 'bp', 'dmv', 'nyu', 'gm', 'lg', 'hp', 'ftc']
manualadd_len =[]
manualadd_word = []
for word in manualadd:
    manualadd_len.append(len(word))
    manualadd_word.append(word)

manual = pd.DataFrame(manualadd_word, columns =['firm'])
manual['cusp_firm_name'] = manualadd_word
manual['length'] = manualadd_len
firmlist = firmlist.append(manual)
firmlist.reset_index(drop = True, inplace = True)

In [6]:
#Count the number of words in each firm name
num_of_words = []
for firm in firmlist.cusp_firm_name:
    num_of_words.append(len(firm.split()))
firmlist['num_of_words'] = num_of_words

In [7]:
#Explode each firm name into a list of words
splits = firmlist.cusp_firm_name.apply(lambda x: x.split())

#Remove words from this expolded list based on the mannualy created list of
#small truckiated words.  This strips these words.
listofwordstostrip = ['c/o', 'd/b/a', 'inc', 'corp', 'dba', 'inc.', 'co', 'llc', 'plc', 'co.', 'corp.', 'pllc', 
                      'pllc.', 'lllc', 'c/o', 'db/a', 'dba.', 'a/ka', 'aka', 'aka.', 'd.b.a.', 'd.b.a', 'p.c.', 
                      'l.l.o.', 'llo', 'p.c.,', 's.c.', 'co.,', 'a/k/a', 'llc.']
firmlist['splits'] = splits
cleaned = [[j for j in i if j not in listofwordstostrip] for i in firmlist.splits]
cleaned_splits = []
for list_of_splits in cleaned:
    cleaned_splits.append(" ".join(list_of_splits))
firmlist['cusp_firm_name'] = cleaned_splits


In [8]:
#Collapse words into lists without whitespace
newFirms = []
for firm in firmlist.cusp_firm_name:
    tempfirm = firm.replace(' ', "")
    newFirms.append(tempfirm)
firmlist['search'] = newFirms

In [9]:
#Drop duplicates and renumber the length column by chareters
firmlist.drop_duplicates(subset = 'cusp_firm_name', inplace = True)
firmlist.reset_index(drop = True, inplace = True)
length = []
for i in firmlist.search:
    templen = len(i)
    length.append(templen)
firmlist['length'] = length

In [10]:
#save firmlist as pickle
firmlist.to_pickle('firmlist.pkl')

In [11]:
#append cusp_firm_name to CFPB
CFPB_lower = []
for word in CFPB.Company:
    CFPB_lower.append(word.lower())
CFPB['lower_company'] = CFPB_lower
maxs_CFPB = CFPB.merge(firmlist, how = 'inner', left_on= 'lower_company', right_on = 'firm')
maxs_CFPB.to_csv('maxs_CFPBv1.csv') 

In [12]:
#append cusp_firm_name to OAG
lower_OAG = silvercase.ENTITY_ORG_NAME.apply(lambda x: str(x).lower())
silvercase['lower_OAG'] = lower_OAG
maxs_OAG = silvercase.merge(firmlist, how = 'inner', left_on= 'lower_OAG', right_on = 'firm')
maxs_OAG.to_csv('maxs_OAGv1.csv')

In [15]:
maxs_OAG.to_pickle('max_OAG_firm_list_v2.pkl')

In [16]:
x= pd.read_pickle('firmlist.pkl')

In [17]:
x

,firm,cusp_firm_name,length,num_of_words,splits,search
0,ticketmaster,ticketmaster,12,1,[ticketmaster],ticketmaster
1,argo real estate llc,argo real estate,14,4,"[argo, real, estate, llc]",argorealestate
2,acura of westchester,acura of westchester,18,3,"[acura, of, westchester]",acuraofwestchester
3,galaxy moving systems,galaxy moving systems,19,3,"[galaxy, moving, systems]",galaxymovingsystems
4,apple company,apple company,12,2,"[apple, company]",applecompany
5,reed elsevier (relx),reed elsevier (relx),18,3,"[reed, elsevier, (relx)]",reedelsevier(relx)
6,"subsidiaries lexisnexis, knowx.com, peoplewise...","subsidiaries lexisnexis, knowx.com, peoplewise...",52,5,"[subsidiaries, lexisnexis,, knowx.com,, people...","subsidiarieslexisnexis,knowx.com,peoplewise.co..."
7,time warner cable,time warner,10,2,"[time, warner]",timewarner
8,juvmfv.com,juvmfv.com,10,1,[juvmfv.com],juvmfv.com
9,tmtfllnlv.com,tmtfllnlv.com,13,1,[tmtfllnlv.com],tmtfllnlv.com
